In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%matplotlib inline
# Python Standard Library Imports
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

print("torch.cuda.device_count()", torch.cuda.device_count()) # torch.cuda.device_count() 2
print("torch.cuda.get_device_name(0)",torch.cuda.get_device_name(0)) # torch.cuda.get_device_name(0)

torch.cuda.device_count() 2
torch.cuda.get_device_name(0) NVIDIA GeForce GTX 1080 Ti


In [12]:
model_name = "MBZUAI/LaMini-Flan-T5-248M"
model_name = "MBZUAI/LaMini-Neo-1.3B"
model_name = "stabilityai/stablelm-3b-4e1t"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.eos_token_id=tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  trust_remote_code=True,
  torch_dtype="auto",
  device_map="auto",
)

model.device

device(type='cuda', index=0)

In [13]:
prompt = """Write a detailed analogy between mathematics and a lighthouse.
Answer:"""

In [15]:
prompt = """
Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestions?

Bob: Have you tried using a timer? It can help you stay on track and avoid distractions.

Alice: That's a good idea. I'll give it a try.

Charlie: Another thing that can help is to break up your study sessions into smaller chunks. It's easier to concentrate on one thing at a time.

Alice: That makes sense. I'll try that too.

Bob: And don't forget to take breaks! It's important to give your brain a rest so you can come back to your studies with a fresh perspective.

Alice: Thanks for the advice, guys. I feel more motivated now.

Charlie: No problem, Alice. We're all in this together.

Bob:"""

In [18]:
prompt = """
hi

hello

how are you?

what new?

hey there"""

In [19]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

input_length = inputs.input_ids.shape[1]

tokens = model.generate(
  **inputs,
  max_new_tokens=32,
  temperature=0.1,
  top_p=0.95,
  do_sample=True,
)

print(tokenizer.decode(tokens[0])[input_length:])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


you?

what new?

hey there

hi

hello

how are you?

what new?

hey there

hi

hello

how are you


In [4]:


inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)

In [7]:
model_inputs = encodeds.to(model.device)

generated_ids = model.generate(
    model_inputs, 
    max_new_tokens=10, 
    do_sample=True,
    num_return_sequences=1,
    temperature=0.2,
)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0]).decode(token)
print(output_str)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 500.00 MiB. GPU 0 has a total capacty of 10.91 GiB of which 464.62 MiB is free. Including non-PyTorch memory, this process has 10.44 GiB memory in use. Of the allocated memory 10.22 GiB is allocated by PyTorch, and 64.32 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = "auto",
    #torch_dtype=torch.float16,
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.eos_token_id=tokenizer.eos_token_id


model = pipeline('text2text-generation', model = model_name, device_map = "auto")
model.device

In [6]:
model_name = "microsoft/phi-1_5"

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", device_map = "auto", trust_remote_code=True)

model.device

device(type='cuda', index=0)

In [32]:
prompt = """Write a detailed analogy between mathematics and a lighthouse.
Answer:"""

prompt = """Write a greeting like 'hi', 'how are you?'.
Answer:"""

prompt = """Human: Hello.\n AI:"""

In [4]:
from transformers import StoppingCriteria

class KeywordsStoppingCriteria(StoppingCriteria):

    """ example usage
    
    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False).to(model.device)
    
    keywords = ['\n','</s>']
    stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, inputs['input_ids'])
    
    outputs = model.generate(
        **inputs, 
        max_new_tokens=20,
        do_sample=True,
        temperature=0.99,
        stopping_criteria=[stopping_criteria],
    )
    """
    
    
    def __init__(self, keywords, tokenizer, input_ids):
        self.keywords = keywords
        self.keyword_ids = []
        self.max_keyword_len = 0
        for keyword in keywords:
            cur_keyword_ids = tokenizer(keyword).input_ids
            if len(cur_keyword_ids) > 1 and cur_keyword_ids[0] == tokenizer.bos_token_id:
                cur_keyword_ids = cur_keyword_ids[1:]
            if len(cur_keyword_ids) > self.max_keyword_len:
                self.max_keyword_len = len(cur_keyword_ids)
            self.keyword_ids.append(torch.tensor(cur_keyword_ids))
        self.tokenizer = tokenizer
        self.start_len = input_ids.shape[1]
    
    def call_for_batch(self, output_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        offset = min(output_ids.shape[1] - self.start_len, self.max_keyword_len)
        self.keyword_ids = [keyword_id.to(output_ids.device) for keyword_id in self.keyword_ids]
        for keyword_id in self.keyword_ids:
            if (output_ids[0, -keyword_id.shape[0]:] == keyword_id).all():
                return True
        outputs = self.tokenizer.batch_decode(output_ids[:, -offset:], skip_special_tokens=True)[0]
        for keyword in self.keywords:
            if keyword in outputs:
                return True
        return False
    
    def __call__(self, output_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        outputs = []
        for i in range(output_ids.shape[0]):
            outputs.append(self.call_for_batch(output_ids[i].unsqueeze(0), scores))
        return all(outputs)
        


NameError: name 'torch' is not defined

In [2]:
inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False).to(model.device)
keywords = ['\n']
stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, inputs['input_ids'])

outputs = model.generate(
    **inputs, 
    max_new_tokens=20,
    do_sample=True,
    temperature=0.99,
    stopping_criteria=[stopping_criteria],
)
input_len = inputs['input_ids'].shape[1]
text = tokenizer.decode(outputs[0,input_len:])
print(text)

NameError: name 'tokenizer' is not defined

In [3]:
outputs[0]

NameError: name 'outputs' is not defined